# Исследовательский анализ данных

Данные по бронированию номеров. Вроде, [отсюда](https://www.kaggle.com/datasets/moro146/hotel-bookings).

Описания там нет. Так что мои предположения:  
|номер|колонка|non-null|тип|описание|
|--|--|--|--|--|
 |2|   lead_time           |            119390 non-null | int64 |  за сколько дней до прибытия был забронирован номер  |
| 20 | assigned_room_type   |           119390 non-null | object |  тип комнаты на самом деле занятый (а не зарезервированный)  |
| 23 | agent                  |         103050 non-null | float64 |  платеж агенту  |
| 24 | company                 |        6797 non-null   | float64 |  компания-агент  |
| 27 | adr                   |          119390 non-null | float64 |  средняя стоимость за ночь  |

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

## Считывание данных

In [3]:
df = pd.read_csv('./data_ignored/hotel_bookings.csv')

In [4]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [5]:
df = df.convert_dtypes()

In [8]:
df.reservation_status_date = df.reservation_status_date.astype('datetime64[ns]')

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           119390 non-null  string        
 1   is_canceled                     119390 non-null  Int64         
 2   lead_time                       119390 non-null  Int64         
 3   arrival_date_year               119390 non-null  Int64         
 4   arrival_date_month              119390 non-null  string        
 5   arrival_date_week_number        119390 non-null  Int64         
 6   arrival_date_day_of_month       119390 non-null  Int64         
 7   stays_in_weekend_nights         119390 non-null  Int64         
 8   stays_in_week_nights            119390 non-null  Int64         
 9   adults                          119390 non-null  Int64         
 10  children                        119386 non-null  Int64  

In [13]:
df.dtypes.unique()

array([string[python], Int64Dtype(), Float64Dtype(), dtype('<M8[ns]')],
      dtype=object)

In [74]:
df.describe(include=int)

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests
count,119390.0,119390.0,119390.0,119390.0,119390.0,119390.0,119390.0,119390.0,119386.0,119390.0,119390.0,119390.0,119390.0,119390.0,103050.0,6797.0,119390.0,119390.0,119390.0
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.10389,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,0.245291,0.792798
min,0.0,0.0,2015.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0
25%,0.0,18.0,2016.0,16.0,8.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,62.0,0.0,0.0,0.0
50%,0.0,69.0,2016.0,28.0,16.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,179.0,0.0,0.0,0.0
75%,1.0,160.0,2017.0,38.0,23.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,229.0,270.0,0.0,0.0,1.0
max,1.0,737.0,2017.0,53.0,31.0,19.0,50.0,55.0,10.0,10.0,1.0,26.0,72.0,21.0,535.0,543.0,391.0,8.0,5.0


Пропуски значений только по агентам - они не всегда же есть.  
Ладно 19 уикэндов провести, но провести 50 рабочих недель без уикендов...  
Проверить еще до 55 взрослых, 10 детей и 10 младенцев в одном номере.  
required_car_parking_spaces до 8. Проверить. Хотя кто-то гостям мог заказать?  
какие номера ждали люди с большими days_in_waiting_list?

In [82]:
df.describe(include=float)

,adr
count,119390.0
mean,101.831122
std,50.53579
min,-6.38
25%,69.29
50%,94.575
75%,126.0
max,5400.0


In [ ]:
Отрицательный adr? Надо почистить.

In [84]:
df.describe(include='string')

,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status
count,119390,119390,119390,118902,119390,119390,119390,119390,119390,119390,119390
unique,2,12,5,177,8,5,10,12,3,4,3
top,City Hotel,August,BB,PRT,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out
freq,79330,13877,92310,48590,56477,97870,85994,74053,104641,89613,75166


Не по всем посетителям есть страна.  
assigned_room_type на 2 больше, чем reserved_room_type

In [80]:
df.describe(include='datetime64')

,reservation_status_date
count,119390
mean,2016-07-30 00:24:47.883407104
min,2014-10-17 00:00:00
25%,2016-02-01 00:00:00
50%,2016-08-07 00:00:00
75%,2017-02-08 00:00:00
max,2017-09-14 00:00:00


## Создание колонок

In [25]:
df['arrival_date'] = pd.to_datetime(
                            df.arrival_date_year.astype(str) 
                            + '-' 
                            + df.arrival_date_month 
                            + '-' + df.arrival_date_day_of_month.astype(str)
                        )

In [27]:
df['arrival_date_month_number'] = df.arrival_date.dt.month

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           119390 non-null  string        
 1   is_canceled                     119390 non-null  Int64         
 2   lead_time                       119390 non-null  Int64         
 3   arrival_date_year               119390 non-null  Int64         
 4   arrival_date_month              119390 non-null  string        
 5   arrival_date_week_number        119390 non-null  Int64         
 6   arrival_date_day_of_month       119390 non-null  Int64         
 7   stays_in_weekend_nights         119390 non-null  Int64         
 8   stays_in_week_nights            119390 non-null  Int64         
 9   adults                          119390 non-null  Int64         
 10  children                        119386 non-null  Int64  

In [108]:
df.groupby(['hotel', 'reserved_room_type']).adr.count()

hotel         reserved_room_type
City Hotel    A                     62595
              B                      1115
              C                        14
              D                     11768
              E                      1553
              F                      1791
              G                       484
              P                        10
Resort Hotel  A                     23399
              B                         3
              C                       918
              D                      7433
              E                      4982
              F                      1106
              G                      1610
              H                       601
              L                         6
              P                         2
Name: adr, dtype: Int64

In [45]:
df.sample(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,arrival_date_month_number
50832,City Hotel,1,135,2016,May,20,9,1,4,2,0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,7,<NA>,0,Transient,79.72,0,1,Canceled,2015-12-27,2016-05-09,5
98987,City Hotel,0,29,2016,October,41,5,0,2,2,0,0,BB,FRA,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,<NA>,0,Transient,152.0,0,2,Check-Out,2016-10-07,2016-10-05,10
35077,Resort Hotel,0,200,2017,April,15,9,2,2,1,2,0,BB,IRL,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,241,<NA>,0,Transient,57.14,0,1,Check-Out,2017-04-13,2017-04-09,4
56801,City Hotel,1,148,2016,September,38,14,0,3,2,0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,86,<NA>,107,Transient-Party,130.0,0,0,Canceled,2016-08-04,2016-09-14,9
98010,City Hotel,0,159,2016,September,39,18,2,3,2,0,0,SC,GBR,Offline TA/TO,TA/TO,0,0,0,A,B,1,No Deposit,79,<NA>,0,Transient,87.72,0,1,Check-Out,2016-09-23,2016-09-18,9
